# Deutscher Wetterdienst

In [1]:
%reset -sf
import pandas as pd

url = "https://opendata.dwd.de/"
path = 'climate_environment/CDC/observations_germany/climate/daily/kl/'
recent_path = path + 'recent/'
historical_path = path + 'historical/'
filename = 'KL_Tageswerte_Beschreibung_Stationen.txt' 
ws = pd.read_csv(url + recent_path + filename, sep="\t", header=0, skiprows = 0, encoding = "ISO-8859-1").dropna()
ws.drop(0, inplace=True)

# save original column names
colnames = ws.columns[0].split(' ')

# rename column for instance to 'dummy'
ws.columns = ['dummy']

# split string up to 6th column
ws = ws['dummy'].str.split('\s+', n=6, expand=True)

# convert 1:6 to numeric
for col in  ws.iloc[:,1:6]:
    ws[col] = pd.to_numeric(ws[col], errors='coerce') 
    
# concat columns back to a type consistent dataframe
wst = pd.concat([ws.iloc[:,0:6], ws[6].str.slice(0,41), ws[6].str.slice(41,)], axis=1)  
wst.columns = colnames

## Read range subset of all weather stations

In [2]:
selection = '83Messstationen.csv'
selws = pd.read_csv(selection, sep=";", header=0, skiprows = 0)
selws.drop(['ICAO-KENNUNG'], axis=1)

,STATIONS-KENNZIFFER,STATIONS_ID,STATIONSNAME,STATIONSHÖHE IN METERN,GEOGR. BREITE,GEOGR. LÄNGE,AUTOMAT SEIT:,BEGINN KLIMAREIHE
0,10501,3,Aachen,202,50° 47',06° 05',01.07.1993,1891
1,10505,15000,Aachen-Orsbach,231,50° 47',06° 01',01.04.2011,2011
2,10291,164,Angermünde,54,53° 01',13° 59',01.11.1991,1947
3,10091,183,Arkona,42,54° 40',13° 26',01.11.1991,1947
4,10852,232,Augsburg,462,48° 25',10° 56',10.11.1996,1947
...,...,...,...,...,...,...,...,...
78,10544,5371,Wasserkuppe,921,50° 29',09° 56',01.05.1995,1936
79,10688,5397,Weiden,440,49° 40',12° 11',04.07.1991,1947
80,10980,5467,Wendelstein,1832,47° 42',12° 00',01.01.1991,1951
81,10655,5705,Würzburg,268,49° 46',09° 57',22.10.1992,1901


## Keep weather stations having observations at least from 1961

In [3]:
refy = 1960
refydf = selws.loc[selws['BEGINN KLIMAREIHE'] <= refy,][['STATIONS_ID','STATIONSNAME']]
refydf

,STATIONS_ID,STATIONSNAME
0,3,Aachen
2,164,Angermünde
3,183,Arkona
4,232,Augsburg
5,282,Bamberg
6,402,Berlin-Dahlem (LFAG)
7,403,Berlin-Dahlem (FU)
9,433,Berlin-Tempelhof
10,691,Bremen
11,722,Brocken


## Download zip file from URL
[howto](https://pythonguides.com/download-zip-file-from-url-using-python/)

In [10]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
from re import compile

def collectRecords(zfile): 
    with BytesIO(zfile.read()) as b, ZipFile(b) as datafile: 
        r = compile("^produkt_klima_tag_.*\.txt$")
        dfound = list(filter(r.match, datafile.namelist()))
        number = len(dfound)
        assert  number == 1, f"WARN: exactly one element expected, got {number} instead"
        rf = datafile.open(dfound[0])
        lines = rf.readlines()
        rf.close()
        header = True
        for bline in lines:
            line = bline.decode('unicode-escape').rstrip('\r\n').split(';')
            del line[-1] # remove last column containing only string 'eor'
            if header: # initialize list of lists
                header = not(header)
                record = [line]
            else:
                record.append(line)
    return record

## Extract observations for weather stations belonging to above subset

In [25]:
zip_url = url + recent_path 
twre = compile(r'tageswerte_KL_[0-9]{5}_akt.zip')
# twre = compile(r'tageswerte_KL_[0-9]{6}_akt.zip')

#zip_url = url + historical_path 
#twre = compile(r'tageswerte_KL_[0-9]{5}_[0-9]{8}_[0-9]{8}_hist.zip')

print(zip_url)
first = True
with urlopen(zip_url) as f:
    for bline in f.readlines():
        zfound = twre.search(bline.decode('utf-8'))
        # print(zfound)
        if zfound:
            zfilename = zfound.string[zfound.start():zfound.end()]
            if refydf['STATIONS_ID'].isin([int(zfilename[14:19])]).any():
                with urlopen(zip_url + zfilename) as z:
                    record = collectRecords(z) # <-- function call   
                    df = pd.DataFrame(record[1:], columns=record[0])
                    if (first):
                        DF = df
                        first = not first
                    else:
                        DF = pd.concat([DF, df]) 

https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/recent/


## Transform data from string to the appropriate type

In [12]:
cols = DF.columns[2:]
DF = DF.replace('-999', '-', regex=True)
DF[cols] = DF[cols].apply(pd.to_numeric, errors='coerce', axis=1)
DF['STATIONS_ID'] = DF['STATIONS_ID'].astype(int)
DF['MESS_DATUM'] = pd.to_datetime(DF['MESS_DATUM'], format='%Y%m%d')
#print(DF.dtypes)
DF

,STATIONS_ID,MESS_DATUM,QN_3,FX,FM,QN_4,RSK,RSKF,SDK,SHK_TAG,NM,VPM,PM,TMK,UPM,TXK,TNK,TGK
0,164,2022-04-23,10.0,10.2,3.3,3.0,0.0,0.0,11.100,0.0,0.9,7.6,1004.56,9.8,65.38,17.8,1.7,-1.7
1,164,2022-04-24,10.0,12.0,4.4,3.0,0.0,0.0,6.850,0.0,6.5,8.2,1000.11,9.2,73.29,17.9,3.8,2.4
2,164,2022-04-25,10.0,7.9,3.7,3.0,0.0,0.0,2.467,0.0,7.5,8.1,1005.29,7.4,79.29,12.3,2.5,-1.0
3,164,2022-04-26,10.0,6.4,1.6,3.0,0.0,0.0,9.550,0.0,5.8,7.0,1010.65,9.3,64.54,16.3,-0.1,-2.5
4,164,2022-04-27,10.0,9.9,3.3,3.0,0.0,0.0,9.417,0.0,3.4,7.5,1016.89,10.1,63.92,17.2,2.9,-0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,5906,2023-10-20,1.0,10.6,3.0,1.0,1.1,6.0,0.267,0.0,7.6,14.9,972.44,15.2,85.88,18.0,13.4,11.5
546,5906,2023-10-21,1.0,17.2,4.8,1.0,0.0,6.0,6.967,0.0,4.5,11.6,986.89,15.3,67.04,18.6,10.6,5.9
547,5906,2023-10-22,1.0,10.1,3.2,1.0,0.0,0.0,3.300,0.0,5.2,10.4,998.90,12.1,74.13,17.3,6.1,3.2
548,5906,2023-10-23,1.0,6.6,1.2,1.0,3.6,6.0,1.383,0.0,5.5,10.8,999.96,9.4,90.79,14.1,3.5,1.2


In [21]:
print(DF['STATIONS_ID'].unique())

[ 164  183  232  282  403  433  691  722  880  891 1270 1346 1358 1420
 1468 1612 1639 1684 1757 1975 2014 2115 2261 2290 2483 2559 2667 2812
 3015 3028 3032 3126 3196 3631 3668 3730 3761 3987 4104 4177 4271 4336
 4466 4625 4887 4911 4931 5100 5371 5397 5516 5705 5792 5906]
